In [ ]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
import tensorflow as tf

print("TensorFlow version: ", tf.__version__)

In [ ]:
train_dir = '/Users/sathvikchowdaryveerapaneni/Desktop/ANIE/DataSet/train'
test_dir = '/Users/sathvikchowdaryveerapaneni/Desktop/ANIE/DataSet/test'




In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

In [ ]:
# Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

# Create the model
model = Model(inputs=vgg_conv.input, outputs=vgg_conv.output)

# Add new layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
dropout = Dropout(0.5)(class1)
output = Dense(1, activation='sigmoid')(dropout)

# define new model
model = Model(inputs=model.inputs, outputs=output)

# summarize
model.summary()
model_path = 'best_model.h5'

In [ ]:
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model_path = 'best_model.h5'

# Create the callback
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [ ]:
# Train the model
with tf.device('/device:GPU:0'):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples/train_generator.batch_size,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples/validation_generator.batch_size,
        verbose=1,
        callbacks=[checkpoint]) 

In [ ]:
# Evaluate the model
scores = model.evaluate_generator(validation_generator)

print("\nAccuracy = ", scores[1])


In [ ]:
# Specify where you want the model to be saved
model.save('First_Model.h5')